# Make pseudobulks

You will run this 2 times, one for cellType pseudobulks and one for granular pseudobulks.
You can edit this in the parameters res_name and celltype_name

# Imports

In [10]:
import sys
from buddi import buddi
from buddi.preprocessing import sc_preprocess


# general imports
import warnings
import numpy as np
import os
import pandas as pd
import scipy as sp
from scipy.sparse import coo_matrix
import collections
import scanpy as sc
import anndata as ad


# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
import sklearn as sk

# matplotlib settings for Jupyter notebooks only
%matplotlib inline

import pickle
import gzip
from pathlib import Path


# Parameters

In [11]:
# parameters

aug_data_path = f"{os.getcwd()}/../../data/single_cell/GSE180661_pseudobulks/"
data_path = f"{os.getcwd()}/../../data/single_cell/GSE180661_processed/"



#####################
### set the study ###
#####################

res_name = "all-granular" # cellType subtype granular

if res_name == "all-cellType":
    celltype_name = "cellType" # cellType subtype celltype_granular
elif res_name == "all-granular":
    celltype_name = "celltype_granular"
elif res_name == "all_subtype":
    celltype_name = "subtype"

in_name = "GSE180661_processed" #HGSCsubtype_processed
processed_sc_file = f"{data_path}/{in_name}.h5ad"



# Load and Process data

### Read in data and metadata

In [12]:
# read in the data

adata = sc.read_h5ad(processed_sc_file)

adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`



In [13]:

# get the columns we need to iterate over for making pseudobulks
adata.obs['scpred_CellType'] = adata.obs[celltype_name].tolist()

adata.obs['stim'] = "CTRL"

# make the gene_ids col
adata.var['gene_ids'] = adata.var.index.tolist()

# we use everything as training
adata.obs["isTraining"] = "Train"

In [14]:
set(adata.obs['scpred_CellType'])

{'immune', 'stromal', 'tumor'}

### look at some data stats

In [15]:
# each sample should only have cells in with "STIM" or "CTRL"
tab = adata.obs.groupby(['sample_id', 'stim']).size()
tab.unstack()

stim,CTRL
sample_id,
SPECTRUM-OV-008_S1_LEFT_PARARENAL_LYMPH_NODE_WITH_COLONIC_MESENTARY,4915
SPECTRUM-OV-009_S1_BOWEL,5232
SPECTRUM-OV-024_S1_INFRACOLIC_OMENTUM,5020
SPECTRUM-OV-036_S1_INFRACOLIC_OMENTUM,5268
SPECTRUM-OV-036_S1_LEFT_ADNEXA,7275
SPECTRUM-OV-049_S1_INFRACOLIC_OMENTUM,7061
SPECTRUM-OV-053_S1_INFRACOLIC_OMENTUM,4264
SPECTRUM-OV-053_S1_LEFT_OVARY,6215
SPECTRUM-OV-053_S1_RIGHT_DIAPHRAGM,5089


In [16]:
# see how many cells per cell type
adata.obs["scpred_CellType"].value_counts()


immune     67828
stromal    29597
tumor      19645
Name: scpred_CellType, dtype: int64

In [17]:
adata.obs.sample_id.unique().to_list()

['SPECTRUM-OV-115_S1_INFRACOLIC_OMENTUM',
 'SPECTRUM-OV-115_S1_LEFT_ADNEXA',
 'SPECTRUM-OV-107_S1_BOWEL',
 'SPECTRUM-OV-110_S1_BOWEL',
 'SPECTRUM-OV-110_S1_INFRACOLIC_OMENTUM',
 'SPECTRUM-OV-083_S1_INFRACOLIC_OMENTUM',
 'SPECTRUM-OV-081_S1_INFRACOLIC_OMENTUM',
 'SPECTRUM-OV-081_S1_RIGHT_ADNEXA',
 'SPECTRUM-OV-075_S1_LEFT_FALLOPIAN_TUBE',
 'SPECTRUM-OV-068_S1_LARGE_BOWEL',
 'SPECTRUM-OV-065_S1_INFRACOLIC_OMENTUM',
 'SPECTRUM-OV-024_S1_INFRACOLIC_OMENTUM',
 'SPECTRUM-OV-008_S1_LEFT_PARARENAL_LYMPH_NODE_WITH_COLONIC_MESENTARY',
 'SPECTRUM-OV-009_S1_BOWEL',
 'SPECTRUM-OV-036_S1_INFRACOLIC_OMENTUM',
 'SPECTRUM-OV-036_S1_LEFT_ADNEXA',
 'SPECTRUM-OV-053_S1_INFRACOLIC_OMENTUM',
 'SPECTRUM-OV-053_S1_RIGHT_DIAPHRAGM',
 'SPECTRUM-OV-053_S1_LEFT_OVARY',
 'SPECTRUM-OV-049_S1_INFRACOLIC_OMENTUM']

# Make pseudobulks

In [18]:
# write out the gene ids
gene_pass = adata.var['gene_ids']
gene_out_file = os.path.join(aug_data_path, f"{res_name}_genes.pkl")
gene_out_path = Path(gene_out_file)
pickle.dump( gene_pass, open( gene_out_path, "wb" ) )

# metadata
sample_order = adata.obs.sample_id.unique().to_list()

stim_order = ['CTRL']
train_order = ['Train']

# now generate all the proportions
total_meta_df = pd.DataFrame(columns = ["sample_id", "stim", "isTraining"])

# no cell noise 
len_vector = adata.obs["scpred_CellType"].unique().shape[0]
cell_noise = [np.random.lognormal(0, 0, adata.var['gene_ids'].shape[0]) for i in range(len_vector)]

# cell type order
cell_order = adata.obs.scpred_CellType.unique()

# simulate different number of cells
num_cells = 200 
idx = 0
for curr_samp in sample_order:
  for curr_stim in stim_order:
      for curr_train in train_order:

        print(f"running {curr_samp} {curr_stim} {curr_train}")


        # make the pseudobulks
        subset_idx = np.logical_and(adata.obs.sample_id == curr_samp, adata.obs.stim == curr_stim)
        subset_idx = np.where(np.logical_and(subset_idx, adata.obs.isTraining == curr_train))[0]
        if len(subset_idx) == 0:
            continue
        
        temp_adata = adata[subset_idx]

        print("make_prop_and_sum")
        prop_df, pseudobulks_df, test_prop_df, test_pseudobulks_df = sc_preprocess.make_prop_and_sum(temp_adata, 
                                                                                num_samples=1000, 
                                                                                num_cells=num_cells,
                                                                                use_true_prop=False,
                                                                                cell_noise=cell_noise,
                                                                                useSampleNoise=False)
        # number of random pseudobulks
        num_rand_pseudo = pseudobulks_df.shape[0] 

        # get the single cell type proportions
        print("get_single_celltype_prop_matrix")
        ct_prop_df = sc_preprocess.get_single_celltype_prop_matrix(num_samp=100, # 100
                                                                    cell_order=cell_order)

        # now get the cell-type specific pseudobulks
        print("use_prop_make_sum")
        prop_df_sc, pseudobulks_df_sc, _ = sc_preprocess.use_prop_make_sum(temp_adata,  
                                                                            num_cells=num_cells, 
                                                                            props_vec=ct_prop_df, 
                                                                            cell_noise=cell_noise,
                                                                            sample_noise=None,
                                                                            useSampleNoise=False)
        # number of random pseudobulks
        num_ct_pseudo = pseudobulks_df_sc.shape[0] 


        # put them together
        print("concat")        
        prop_df = pd.concat([prop_df,prop_df_sc])
        pseudobulks_df = pd.concat([pseudobulks_df, pseudobulks_df_sc])

        # make the metadata
        num_samps = pseudobulks_df.shape[0] 
        samp_type = ["bulk"]*num_samps
        cell_prop_type = ["random"]*num_rand_pseudo+["cell_type_specific"]*num_ct_pseudo 
        samp_type = ["sc_ref"]*(num_rand_pseudo+num_ct_pseudo)
        
        metadata_df = pd.DataFrame(data = {"sample_id":[curr_samp]*num_samps, 
                                          "stim":[curr_stim]*num_samps,
                                          "isTraining":[curr_train]*num_samps,
                                          "cell_prop_type":cell_prop_type,
                                          "samp_type":samp_type,})

        # make the proportions instead of cell counts
        prop_df = prop_df.div(prop_df.sum(axis=1), axis=0)
        pseudobulk_file = os.path.join(aug_data_path, f"{res_name}_{curr_samp}_{curr_stim}_{curr_train}_pseudo_splits.pkl")
        prop_file = os.path.join(aug_data_path, f"{res_name}_{curr_samp}_{curr_stim}_{curr_train}_prop_splits.pkl")
        meta_file = os.path.join(aug_data_path, f"{res_name}_{curr_samp}_{curr_stim}_{curr_train}_meta_splits.pkl")

        print("write")        
        pseudobulk_path = Path(pseudobulk_file)
        prop_path = Path(prop_file)
        meta_path = Path(meta_file)
        pickle.dump( prop_df, open( prop_path, "wb" ) )
        pickle.dump( pseudobulks_df, open( pseudobulk_path, "wb" ) )
        pickle.dump( metadata_df, open( meta_path, "wb" ) )





running SPECTRUM-OV-115_S1_INFRACOLIC_OMENTUM CTRL Train
make_prop_and_sum
0
100
200
300
400
500
600
700
800
900
1000
get_single_celltype_prop_matrix
use_prop_make_sum
0
100
200
concat
write
running SPECTRUM-OV-115_S1_LEFT_ADNEXA CTRL Train
make_prop_and_sum
0
100
200
300
400
500
600
700
800
900
1000
get_single_celltype_prop_matrix
use_prop_make_sum
0
100
200
concat
write
running SPECTRUM-OV-107_S1_BOWEL CTRL Train
make_prop_and_sum
0
100
200
300
400
500
600
700
800
900
1000
get_single_celltype_prop_matrix
use_prop_make_sum
0
100
200
concat
write
running SPECTRUM-OV-110_S1_BOWEL CTRL Train
make_prop_and_sum
0
100
200
300
400
500
600
700
800
900
1000
get_single_celltype_prop_matrix
use_prop_make_sum
0
100
200
concat
write
running SPECTRUM-OV-110_S1_INFRACOLIC_OMENTUM CTRL Train
make_prop_and_sum
0
100
200
300
400
500
600
700
800
900
1000
get_single_celltype_prop_matrix
use_prop_make_sum
0
100
200
concat
write
running SPECTRUM-OV-083_S1_INFRACOLIC_OMENTUM CTRL Train
make_prop_and_sum
0
1